In [1]:
import tensorflow as tf
import numpy as np
import math

Hint: Use dtype=tf.float64 if you want to have same precision as numpy for testing<br>
Hint: You migth wanna use tf.InterativeSession for convenience

1a: Create two random 0-d tensors x and y of any distribution. <br>
Create a TensorFlow object that returns x + y if x > y, and x - y otherwise. <br>
Hint: look up tf.cond() <br>
I do the first problem for you <br>

In [2]:
def task_1a_np(x, y):
    return np.where(x > y, x + y, x - y)

In [3]:
X = tf.placeholder(tf.float64, name="X")
Y = tf.placeholder(tf.float64, name="Y")
out = tf.cond(X > Y, lambda: X + Y, lambda: X - Y)

In [4]:
with tf.Session() as sess:
    for xx, yy in np.random.uniform(size=(50, 2)):
        actual = sess.run(out, feed_dict={X:xx, Y:yy})
        expected = task_1a_np(xx, yy)
        if actual != expected:
            print('Fail')
    else:
        print('Success')

Success


1b: Create two 0-d tensors x and y randomly selected from the range [-1, 1).<br>
Return x + y if x < y, x - y if x > y, 0 otherwise.<br>
Hint: Look up tf.case().<br>

In [5]:
def task_1b_np(x, y):
    return np.select(condlist=[x < y, x > y],
                     choicelist=[x + y, x - y],
                     default=0)

In [6]:
X = tf.placeholder(tf.float64, name="X")
Y = tf.placeholder(tf.float64, name="Y")
out = tf.case([(X > Y, lambda: X - Y), (X < Y, lambda: X + Y)],
    default=lambda: tf.constant(0.0, tf.float64))

In [7]:
with tf.Session() as sess:
    for xx, yy in np.random.uniform(low=-1.0, size=(50, 2)):
        actual = sess.run(out, feed_dict={X:xx, Y:yy})
        expected = task_1b_np(xx, yy)
        if actual != expected:
            print('Fail')
    else:
        print('Success')

Success


1c: Create the tensor x of the value [[0, -2, -1], [0, 1, 2]] <br>
and y as a tensor of zeros with the same shape as x. <br>
Return a boolean tensor that yields Trues if x equals y element-wise. <br>
Hint: Look up tf.equal(). <br>

In [8]:
def task_1c_np():
    x = np.array([[0, -2, -1], [0, 1, 2]])
    y = np.zeros_like(x)
    return x == y

In [9]:
X = tf.constant([[0, -2, -1], [0, 1, 2]], tf.float64, name="X")
Y = tf.zeros([2, 3], tf.float64, name="Y")
out = tf.equal(X, Y)

In [10]:
session = tf.InteractiveSession()
print('Success' if np.all(out.eval() == task_1c_np()) else 'Fail')
session.close()

Success


1d:<br>
Get the indices of elements in x whose values are greater than 30.<br>
Hint: Use tf.where().<br>
Then extract elements whose values are greater than 30.<br>
Hint: Use tf.gather().<br>

In [11]:
def task_1d_np(x):
    return x[x > 30].reshape(-1, 1)

In [12]:
X = tf.placeholder(tf.float64, shape=(10, 2), name="X")
ind = tf.where(X > 30, name="indexes")
out = tf.reshape(tf.gather_nd(X, ind), [-1, 1])

In [13]:
with tf.Session() as sess:
    xx = np.random.uniform(high=50, size=(10, 2))
    actual = sess.run(out, feed_dict={X:xx})
    expected = task_1d_np(xx)
    
    if np.any(actual != expected):
        print('Fail')
    else:
        print('Success')

Success


1e: Create a diagnoal 2-d tensor of size 6 x 6 with the diagonal values of 1,<br>
2, ..., 6<br>
Hint: Use tf.range() and tf.diag().<br>

In [14]:
def task_1e_np():
    return np.diag(np.arange(1, 7))

In [15]:
out = tf.diag(tf.range(1, 7))

In [16]:
session = tf.InteractiveSession()
print('Success' if np.all(out.eval() == task_1e_np()) else 'Fail')
session.close()

Success


1f: Create a random 2-d tensor of size 10 x 10 from any distribution.<br>
Calculate its determinant.<br>
Hint: Look at tf.matrix_determinant().<br>

In [17]:
def task_1f_np(x):
    return np.linalg.det(x)

In [18]:
X = tf.placeholder(tf.float64, shape=(10, 10), name="X")
out = tf.matrix_determinant(X)

In [19]:
with tf.Session() as sess:
    xx = np.random.uniform(size=(10, 10))
    
    actual = sess.run(out, feed_dict={X:xx})
    expected = task_1f_np(xx)
    
    if abs(actual - expected) > 1e-9:
        print('Fail')
    else:
        print('Success')

Success


1g: Create tensor x with value [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9].<br>
Return the unique elements in x<br>
Hint: use tf.unique(). Keep in mind that tf.unique() returns a tuple.<br>

In [20]:
def task_1g_np():
    x = [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9]
    _, idx = np.unique(x, return_index=True)
    return np.take(x, sorted(idx))

In [21]:
X = tf.constant([5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9], name="X")
y, idx = tf.unique(X)

In [22]:
session = tf.InteractiveSession()
print('Success' if np.all(y.eval() == task_1g_np()) else 'Fail')
session.close()

Success


1h: Create two tensors x and y of shape 300 from any normal distribution,<br>
as long as they are from the same distribution.<br>
Use tf.cond() to return:<br>
- The mean squared error of (x - y) if the average of all elements in (x - y)<br>
  is negative, or<br>
- The sum of absolute value of all elements in the tensor (x - y) otherwise.<br>
Hint: see the Huber loss function in the lecture slides 3.<br>

In [23]:
def task_1h_np(x, y):
    average = np.mean(x - y)
    mse = np.mean((x - y) ** 2)
    asum = np.sum(np.abs(x - y))
    return mse if average < 0 else asum

In [24]:
X = tf.placeholder(tf.float64, shape=300, name="X")
Y = tf.placeholder(tf.float64, shape=300, name="Y")
residual = X - Y
mean = tf.reduce_mean(residual, name="mean")
mse = tf.reduce_mean(tf.square(residual), name="mse")
out = tf.where(mean < 0, mse, tf.reduce_sum(tf.abs(residual)))

In [25]:
with tf.Session() as sess:
    xx = np.random.normal(size=300)
    yy = np.random.normal(size=300)
    
    actual = sess.run(out, feed_dict={X:xx, Y:yy})
    expected = task_1h_np(xx, yy)
    
    if abs(actual - expected) > 1e-9:
        print('Fail')
    else:
        print('Success')

Success
